In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.regularizers import l2 
keras.__version__

In [2]:
# confirm Keras sees the GPU (for TensorFlow 1.X + Keras)
from keras import backend
assert len(backend.tensorflow_backend._get_available_gpus()) > 0
print(backend.tensorflow_backend._get_available_gpus())





['/job:localhost/replica:0/task:0/device:GPU:0']


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10163029109895009842
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5776776731185618610
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4783588398416312767
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7511012148
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10788100613164667047
physical_device_desc: "device: 0, name: Tesla M60, pci bus id: 0000:00:1e.0, compute capability: 5.2"
]


In [4]:
import pandas as pd
import numpy as np
import glob
import cv2
# stack overflow example taken from: https://stackoverflow.com/questions/37747021/create-numpy-array-of-images
X_data = []
y = []

files = glob.glob ("../.././cell_images/Parasitized/*.png")
for myFile in files:
    image = cv2.imread(myFile,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(128,128))
    X_data.append(image)
    y.append((int)(1))
files = glob.glob ("../.././cell_images/Uninfected/*.png")
for myFile in files:
    image = cv2.imread(myFile,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(128,128))
    X_data.append (image)
    y.append((int)(0))
npArr =np.array(X_data)
print('X_data shape:', np.array(X_data).shape)

X_data shape: (27558, 128, 128, 3)


In [5]:
print(npArr[0].shape)
print(npArr[1].shape)
pixels = npArr
pixels.shape

(128, 128, 3)
(128, 128, 3)


(27558, 128, 128, 3)

In [6]:
y = np.array(y)
h=128
w=128
print(y)

[1 1 1 ... 0 0 0]


In [ ]:
from matplotlib import pyplot as plt

# a helper plotting function

# def plot_gallery(images, titles, h, w,n_row=3, n_col=6):
#     """Helper function to plot a gallery of portraits"""
#     plt.figure(figsize=(1.7 * n_col, 2.3 * n_row))
#     plt.subplots_adjust(bottom=0, left=.01, right=.99, top=.90, hspace=.35)
#     for i in range(n_row * n_col):
#         plt.subplot(n_row, n_col, i + 1)
#         plt.imshow(images[i])
#         plt.title(titles[i], size=12)
#         plt.xticks(())
#         plt.yticks(())
#     plt.show()
# plot_gallery(x_train, y_train, h, w) # defaults to showing a 3 by 6 subset of the faces
print(y[0])
plt.imshow(pixels[0])
plt.show()

In [8]:
from sklearn.model_selection import train_test_split
import keras.utils
from keras import utils as np_utils
x_train,x_test,y_train,y_test = train_test_split(pixels, y, test_size = 1/5, random_state = 1)
NUM_CLASSES = 2
y_train_ohe = keras.utils.to_categorical(y_train, 2)
y_test_ohe = keras.utils.to_categorical(y_test, 2)
print(x_train[:4000].shape)
print(y_train_ohe[:4000].shape)
x_train = x_train/255.0 - 0.5
x_test = x_test/255.0 - 0.5

(4000, 128, 128, 3)
(4000, 2)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=360, # used, Int. Degree range for random rotations.
    width_shift_range=0.05, # used, Float (fraction of total width). Range for random horizontal shifts.
    height_shift_range=0.05, # used,  Float (fraction of total height). Range for random vertical shifts.
    shear_range=0., # Float. Shear Intensity (Shear angle in counter-clockwise direction as radians)
    zoom_range=0.,
    channel_shift_range=0.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None)
datagen.fit(x_train)

In [ ]:
tmps = datagen.flow(x_train, y_train, batch_size=5000)
for tmp in tmps:
    x_train = np.concatenate((x_train,tmp[0]))
    y_train = np.concatenate((y_train,tmp[1]))
    break
print(x_train.shape)

In [9]:
x_train_flat = []
x_test_flat = []
for val in x_train:
    x_train_flat.append(np.ravel(val))
x_train_flat = np.array(x_train_flat)
for val in x_test:
    x_test_flat.append(np.ravel(val))
x_test_flat = np.array(x_test_flat)

## Ensemble Style CNN

In [ ]:
cnn2 = Sequential()
cnn2.add( Conv2D(filters=16, 
                kernel_size=(3,3), 
                input_shape = (128,128,3),
                padding='same', 
                activation='linear',data_format="channels_last") ) # more compact syntax

num_filt_layers = [32, 64]
for num_filters in num_filt_layers:
    cnn2.add( Conv2D(filters=num_filters, 
                    kernel_size=(3,3), 
                    padding='same', 
                    activation='relu',data_format="channels_last") ) # more compact syntax

    # max pooling
    cnn2.add( MaxPooling2D(pool_size=(2, 2), data_format="channels_last") )
    

# add one layer on flattened output
cnn2.add(Dropout(0.25))
cnn2.add(Flatten())
cnn2.add(Dense(128, activation='relu'))
cnn2.add(Dropout(0.5))
cnn2.add(Dense(64, activation='relu'))
cnn2.add(Dense(1, activation='sigmoid'))

# Let's train the model 
cnn2.compile(loss='binary_crossentropy', # 'categorical_crossentropy' 'mean_squared_error'
              optimizer='rmsprop', # 'adadelta' 'rmsprop'
              metrics=['accuracy'])
cnn2.summary()

In [ ]:
history = cnn2.fit(x_train, y_train, 
        batch_size=64, epochs=10, 
        shuffle=True, verbose=1,
        validation_data=(x_test,y_test))

plt.figure(figsize=(10,4))
plt.subplot(2,2,1)
plt.plot(history.history['acc'])

plt.ylabel('Accuracy %')
plt.title('Training')
plt.subplot(2,2,2)
plt.plot(history.history['val_acc'])
plt.title('Validation')

plt.subplot(2,2,3)
plt.plot(history.history['loss'])
plt.ylabel('MSE Training Loss')
plt.xlabel('epochs')

plt.subplot(2,2,4)
plt.plot(history.history['val_loss'])
plt.xlabel('epochs')
plt.show()

## Alexnet style CNN

In [10]:
%%time


cnn = Sequential()
# let's start with an AlexNet style convolutional phase
cnn.add(Conv2D(filters=32,
                kernel_size=(3,3),
                input_shape = (128,128,3),
                padding='same', 
                activation='linear',data_format="channels_last")) # more compact syntax
# cnn.add(MaxPooling2D(pool_size=(2, 2)))
# no max pool before next conv layer!!
cnn.add(Conv2D(filters=64,
                kernel_size=(3,3), 
                padding='same', 
                activation='relu',data_format="channels_last")) # more compact syntax
cnn.add(MaxPooling2D(pool_size=(2, 2)))
    

# add one layer on flattened output
cnn.add(Dropout(0.25))
cnn.add(Flatten())
cnn.add(Dense(128, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))

# Let's train the model 
cnn.compile(optimizer='adam',
              loss='binary_crossentropy', # 'adadelta' 'rmsprop'
              metrics=['accuracy'])
cnn.summary()



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 262144)            0         
______________________________________________________________

In [11]:
# we need to exapnd the dimensions here to give the 
#   "channels" dimension expected by Keras
# cnn.fit(x_train, y_train, 
#         batch_size=32, epochs=30, verbose=1,
#         validation_data=(x_test,y_test),
#         callbacks=[EarlyStopping(monitor='val_loss', patience=3)])
history =  cnn.fit(x_train, y_train, 
        batch_size=64, epochs=10, 
        shuffle=True, verbose=1,
        validation_data=(x_test,y_test))
plt.figure(figsize=(10,4))
plt.subplot(2,2,1)
plt.plot(history.history['acc'])

plt.ylabel('Accuracy %')
plt.title('Training')
plt.subplot(2,2,2)
plt.plot(history.history['val_acc'])
plt.title('Validation')

plt.subplot(2,2,3)
plt.plot(history.history['loss'])
plt.ylabel('MSE Training Loss')
plt.xlabel('epochs')

plt.subplot(2,2,4)
plt.plot(history.history['val_loss'])
plt.xlabel('epochs')
plt.show()

Train on 22046 samples, validate on 5512 samples
Epoch 1/10
14080/22046 [==================>...........] - ETA: 22s - loss: 0.6133 - acc: 0.6995

KeyboardInterrupt: 

In [ ]:
case1 = 0.0 # both models are correct
case2 = 0.0 # MLP correct, NN incorrect
case3 = 0.0 # NN correct, MPL incorrect
case4 = 0.0 # Both incorrect
yhat1 = np.round(cnn.predict(x_test))
yhat2 = np.round(cnn2.predict(x_test))
for i, value in enumerate(y_test):
    if ((value == yhat1[i]) and (value == yhat2[i])):
        case1 += 1
    elif ((value == yhat1[i]) and (value != yhat2[i])):
        case2 += 1
    elif ((value != yhat1[i]) and (value == yhat2[i])):
        case3 += 1
    elif ((value != yhat1[i]) and (value != yhat2[i])):
        case4 += 1
    else:
        print(value, yhat2[i], yhat1[i])
case1 /= 6
case2 /= 6
case3 /= 6
case4 /= 6
print("McNemar test between Alexnet and Custom CNN")
print(np.round(case1), " | ", np.round(case2))
print(np.round(case3), " | ", np.round(case4))

In [ ]:
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn import metrics
yhat1prob = cnn.predict(x_test)
yhat2prob = cnn2.predict(x_test)
print(yhat1prob.ravel())
print(y_test)
fpr_keras, tpr_keras, thresholds = metrics.roc_curve(y_test, yhat1prob.ravel(), pos_label=1)
auc_keras = auc(fpr_keras, tpr_keras)
fpr_keras2, tpr_keras2, thresholds2 = metrics.roc_curve(y_test, yhat2prob.ravel(), pos_label=1)
auc_keras2 = auc(fpr_keras2, tpr_keras2)
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Alexnet (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_keras2, tpr_keras2, label='Custom CNN (area = {:.3f})'.format(auc_keras2))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()
# Zoom in view of the upper left corner.
plt.figure(2)
plt.xlim(0, 0.2)
plt.ylim(0.8, 1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Alexnet (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_keras2, tpr_keras2, label='Custom CNN (area = {:.3f})'.format(auc_keras2))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve (zoomed in at top left)')
plt.legend(loc='best')
plt.show()

In [ ]:
%%time
print(x_train_flat.shape[1])
# make a 3 layer keras MLP
mlp = Sequential()
mlp.add( Dense(input_dim=x_train_flat.shape[1], units=30, activation='relu') )
mlp.add( Dense(units=15, activation='relu') )
mlp.add( Dense(1) )
mlp.add( Activation('softmax') )

mlp.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['accuracy'])

mlp.fit(x_train_flat, y_train, 
        batch_size=128, epochs=15, 
        shuffle=True, verbose=1)

In [ ]:
from sklearn import metrics as mt
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

def compare_mlp_cnn(cnn, mlp, X_test, y_test):
    plt.figure(figsize=(15,5))
    if cnn is not None:
        yhat_cnn = np.argmax(cnn.predict(X_test), axis=1)
        acc_cnn = mt.accuracy_score(y_test,yhat_cnn)
        plt.subplot(1,2,1)
        cm = mt.confusion_matrix(y_test,yhat_cnn)
        cm = cm/np.sum(cm,axis=1)[:,np.newaxis]
        sns.heatmap(cm, annot=True, fmt='.2f')
        plt.title('CNN: '+str(acc_cnn))
    
    if mlp is not None:
        yhat_mlp = np.argmax(mlp.predict(X_test), axis=1)
        acc_mlp = mt.accuracy_score(y_test,yhat_mlp)
        plt.subplot(1,2,2)
        cm = mt.confusion_matrix(y_test,yhat_mlp)
        cm = cm/np.sum(cm,axis=1)[:,np.newaxis]
        sns.heatmap(cm,annot=True, fmt='.2f')
        plt.title('MLP: '+str(acc_mlp))

In [ ]:
 compare_mlp_cnn(cnn, mlp, x_test, y_test)